In [1]:
import pandas as pd
import requests as r

#handle weird kaleido error
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])

from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [2]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2_Fees' in pwd:
    prepend = ''
else:
    prepend = 'L2_Fees/'

In [3]:
trailing_num_days = 365#90

start_date = date.today()-timedelta(days=trailing_num_days +1)

start_date = max(start_date, date(2021,11,12))
print(start_date)
# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

2021-11-20


In [4]:
eth_prices = r.get(
                'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days='
                + str(trailing_num_days)
                + '&interval=daily'
                )
ethp = pd.DataFrame(eth_prices.json()['prices'])
ethp.rename(columns={0:'date',1:'eth_price'},inplace=True)
ethp['date'] = pd.to_datetime(ethp['date'], unit ='ms')
ethp['date'] = ethp['date'].dt.strftime("%Y-%m-%d")
ethp.drop_duplicates(subset=['date'],inplace=True,keep='first')

In [5]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)

In [6]:
async def get_cryptostats_api(api_core_string,dt_string ):
        dt_string = dt_string.strftime("%Y-%m-%d")
        # print(dt_string)
        i = 0
        res = pd.DataFrame()
        # #try 5 times
        # while i < 5:
        #         try: 
        # async with aiohttp.ClientSession() as session:
        retry_client = RetryClient()
        async with retry_client.get(api_core_string + dt_string, retry_options=ExponentialRetry(attempts=10), raise_for_status=statuses) as response:
                try:
                        prot_req = await response.json()
                        res = pd.json_normalize(prot_req['data']).reset_index()
                        res['date'] = dt_string
                except Exception as e:
                        raise Exception("Could not convert json")
        await retry_client.close()
                
                # prot_req = await retry_client.get(api_core_string + dt_string, headers=header)
                # prot_req = await prot_req.json()
        
                # except:
                #         i = i+1
                #         time.sleep(1)
                #         continue
                # break
        return res

# async def gather_with_concurrency(n, *tasks):
#     semaphore = asyncio.Semaphore(n)
#     async def sem_task(task):
#         async with semaphore:
#             return await task
        

In [7]:
# ALL Relevant API Strings
# https://cryptostats.community/discover/fees
# This is dependent on crypto stats, eventually maybe we mod this to pull from TheGraph, but this is simplest por ahora
# https://docs.cryptostats.community/tutorial/how-to-query-with-rest-api/making-get-requests
        #How to make a single query
        #To make a single query on a collection. Send a GET request to
        #https://api.cryptostats.community/api/v1/<collection-id>/<query>/<arg>
        
        # How to make multiple queries
        # To make multiple queries on a collection, send a GET request to
        # https://api.cryptostats.community/api/v1/<collection-id>/<query-1>,<query-2>/<query-1-arg>,<query-2-arg>

api_core = 'https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/' #Can only pull everything all at once. Odd.
date_rng = pd.date_range(start=start_date, end=date.today()-timedelta(days=1))
date_rng_str = ','.join(x.strftime("%Y-%m-%d") for x in date_rng)
# print(date_rng[0].strftime("%Y-%m-%d"))

In [8]:
def get_range(date_range):
        data_dfs = []
        fee_df = []
        # for dt in date_range:
        #         await asyncio.gather()
        #         data_dfs.append(res_df)
        #         # res.columns
        # try:
        #         loop.close()
        # except:
        #         #nothing
        loop = asyncio.get_event_loop()
        tasks = [get_cryptostats_api(api_core,dt) for dt in date_range]
        # print(tasks)
        data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
        # print(len(data_dfs))
        data_dfs = [value for value in data_dfs if type(value) == pd.DataFrame]
        # print(len(data_dfs))
        # print(date_range)
        # loop.close()
        # print(data_dfs)
        fee_df = pd.concat(data_dfs)
        return fee_df

In [9]:


fdf = get_range(date_rng)
# r.get('https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/2022-05-21')
# print(r)

Aiohttp retry client was not closed
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fce6880e1c0>


In [10]:
fdf['results.oneDayTotalFees'] = fdf['results.oneDayTotalFees'].fillna(0)
# display(fdf)

In [11]:
data_fdf = fdf[['id','metadata.blockchain','results.oneDayTotalFees','metadata.name','metadata.category','date']]
data_fdf = data_fdf.merge(ethp,on='date',how='left')

data_fdf = data_fdf.groupby(['date','metadata.blockchain','metadata.name']).sum()
data_fdf.reset_index(inplace=True)
# Exclude the chain's fees
data_fdf = data_fdf[data_fdf['metadata.blockchain'] != data_fdf['metadata.name']]

data_fdf['oneDayTotalFees_ETH'] = data_fdf['results.oneDayTotalFees']/data_fdf['eth_price']


data_fdf

,date,metadata.blockchain,metadata.name,results.oneDayTotalFees,eth_price,oneDayTotalFees_ETH
0,2021-11-20,Arbitrum One,Aave,0.000000,0.000000,NaN
1,2021-11-20,Arbitrum One,Abracadabra.money,6661.584981,0.000000,inf
3,2021-11-20,Arbitrum One,Balancer,11263.673183,0.000000,inf
4,2021-11-20,Arbitrum One,Curve,6548.034312,0.000000,inf
5,2021-11-20,Arbitrum One,GMX,87487.366734,0.000000,inf
...,...,...,...,...,...,...
30654,2022-11-20,Polygon,SushiSwap,2555.396920,1218.893657,2.096489
30655,2022-11-20,Polygon,Uniswap,64195.750072,1218.893657,52.667228
30657,2022-11-20,Terra,Terraswap,0.000000,1218.893657,0.000000
30658,2022-11-20,Velas,Kyberswap,0.000000,1218.893657,0.000000


In [12]:
data_fdf_op = data_fdf[data_fdf['metadata.blockchain'] == 'Optimism']
data_fdf_op['chain_app_id'] = data_fdf_op['metadata.blockchain'] + '_' + data_fdf_op['metadata.name']

data_fdf_op.sort_values(by =['date','results.oneDayTotalFees','metadata.blockchain'], ascending = [True,False,False],inplace = True)
data_fdf_op.reset_index(inplace=True,drop=True)

data_fdf_op['fees_7_day_avg'] = data_fdf_op.groupby('chain_app_id')['results.oneDayTotalFees'].rolling(7,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_op['fees_30_day_avg'] = data_fdf_op.groupby('chain_app_id')['results.oneDayTotalFees'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_op['fees_90_day_avg'] = data_fdf_op.groupby('chain_app_id')['results.oneDayTotalFees'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)

data_fdf_op['fees_7_day_avg_eth'] = data_fdf_op.groupby('chain_app_id')['oneDayTotalFees_ETH'].rolling(7,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_op['fees_30_day_avg_eth'] = data_fdf_op.groupby('chain_app_id')['oneDayTotalFees_ETH'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_op['fees_90_day_avg_eth'] = data_fdf_op.groupby('chain_app_id')['oneDayTotalFees_ETH'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)


# display(data_fdf_op)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_27624/300847970.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_27624/300847970.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_27624/300847970.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,date,metadata.blockchain,metadata.name,results.oneDayTotalFees,eth_price,oneDayTotalFees_ETH,chain_app_id,fees_7_day_avg,fees_30_day_avg,fees_90_day_avg,fees_7_day_avg_eth,fees_30_day_avg_eth,fees_90_day_avg_eth
0,2021-11-20,Optimism,Hop Protocol,17443.487923,0.000000,inf,Optimism_Hop Protocol,17443.487923,17443.487923,17443.487923,NaN,NaN,NaN
1,2021-11-20,Optimism,Synthetix,15429.037100,0.000000,inf,Optimism_Synthetix,15429.037100,15429.037100,15429.037100,NaN,NaN,NaN
2,2021-11-20,Optimism,Uniswap,13145.823544,0.000000,inf,Optimism_Uniswap,13145.823544,13145.823544,13145.823544,NaN,NaN,NaN
3,2021-11-20,Optimism,Aave,0.000000,0.000000,NaN,Optimism_Aave,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,2021-11-20,Optimism,Curve,0.000000,0.000000,NaN,Optimism_Curve,0.000000,0.000000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2022-11-20,Optimism,Hop Protocol,3041.430197,1218.893657,2.495238,Optimism_Hop Protocol,3001.168510,2992.653494,2349.375934,2.448715,2.202041,1.660497
2916,2022-11-20,Optimism,Aave,2843.124689,1218.893657,2.332545,Optimism_Aave,3185.376459,23910.125274,65591.160719,2.601047,16.357761,44.806469
2917,2022-11-20,Optimism,Curve,2193.078187,1218.893657,1.799237,Optimism_Curve,2102.648774,2594.453719,1779.751040,1.721262,1.997463,1.295863
2918,2022-11-20,Optimism,Kyberswap,1661.173714,1218.893657,1.362854,Optimism_Kyberswap,673.271992,520.167693,236.769811,0.551894,0.384581,0.175895


In [15]:
# Get Fees by app in a time period
app_name = 'Aave'
chain_name = 'Optimism'
start_date_name = date(2022,8,4)
end_date_name = date(2022,11,4)
filter_df = data_fdf_op[
                (data_fdf_op['metadata.name'] == app_name) 
                & (data_fdf_op['metadata.blockchain'] == chain_name)
                & (pd.to_datetime(data_fdf_op['date']) >= pd.to_datetime(start_date_name) )
                & (pd.to_datetime(data_fdf_op['date']) <= pd.to_datetime(end_date_name) )
                ]

In [17]:
# display(filter_df)
# print(filter_df['results.oneDayTotalFees'].sum())

,date,metadata.blockchain,metadata.name,results.oneDayTotalFees,eth_price,oneDayTotalFees_ETH,chain_app_id,fees_7_day_avg,fees_30_day_avg,fees_90_day_avg,fees_7_day_avg_eth,fees_30_day_avg_eth,fees_90_day_avg_eth
2050,2022-08-04,Optimism,Aave,36339.129246,1621.341324,22.413004,Optimism_Aave,5449.496461,1487.268138,637.108895,3.355355,0.942957,0.411516
2056,2022-08-05,Optimism,Aave,117122.099694,1612.418907,72.637513,Optimism_Aave,22141.110401,5385.351447,1938.164320,13.708868,3.358928,1.218488
2064,2022-08-06,Optimism,Aave,141414.487928,1725.382627,81.961233,Optimism_Aave,42303.978459,10090.070371,3509.167346,25.395074,6.083316,2.129066
2072,2022-08-07,Optimism,Aave,142997.714768,1693.296592,84.449302,Optimism_Aave,62693.229867,14850.681622,5097.598485,37.436279,8.893472,3.067220
2081,2022-08-08,Optimism,Aave,150667.221956,1699.006418,88.679607,Optimism_Aave,84172.207108,19862.666025,6771.094662,50.078093,11.841144,4.052289
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2754,2022-10-31,Optimism,Aave,21721.481945,1591.051773,13.652279,Optimism_Aave,69119.617696,60394.378005,91112.009927,45.412727,44.325778,58.984427
2762,2022-11-01,Optimism,Aave,15352.969801,1572.979515,9.760438,Optimism_Aave,63559.022869,58512.351935,91278.067604,41.034964,42.825565,59.090110
2770,2022-11-02,Optimism,Aave,10237.353998,1580.378733,6.477785,Optimism_Aave,55658.162039,56511.494260,90988.047879,35.556103,41.208718,58.913052
2779,2022-11-03,Optimism,Aave,5299.224254,1520.638364,3.484868,Optimism_Aave,44957.887330,54402.384223,89745.571486,28.750468,39.597836,58.144690


8235723.756261774


In [ ]:
data_fdf_chain = data_fdf.groupby(['date','metadata.blockchain']).sum()
data_fdf_chain.reset_index(inplace=True)

data_fdf_chain = data_fdf_chain[data_fdf_chain['metadata.blockchain'] == 'Optimism']
data_fdf_chain.reset_index(inplace=True,drop=True)

data_fdf_chain.sort_values(by =['date','results.oneDayTotalFees'], ascending = [True,False],inplace = True)
data_fdf_chain['fees_30_day_avg'] = data_fdf_chain.groupby('metadata.blockchain')['results.oneDayTotalFees'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_chain['fees_90_day_avg'] = data_fdf_chain.groupby('metadata.blockchain')['results.oneDayTotalFees'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)

data_fdf_chain['fees_30_day_avg_eth'] = data_fdf_chain.groupby('metadata.blockchain')['oneDayTotalFees_ETH'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_chain['fees_90_day_avg_eth'] = data_fdf_chain.groupby('metadata.blockchain')['oneDayTotalFees_ETH'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)


#compound monthly growth rates
# https://velawoodlaw.com/glossary-term/compounded-monthly-growth-rate-cmgr/#:~:text=Compounded%20Monthly%20Growth%20Rate%20(CMGR)%20is%20a%20calculation%20that%20helps,of%20Months)%20-1%5D.
# CMGR = (Latest Month/ First Month)^(1/# of Months) -1].
data_fdf_chain['fees_1q_cmgr'] = (
                                data_fdf_chain['fees_30_day_avg']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg'].shift(30*3*1)
                                ) ** (1/(3*1)) - 1

data_fdf_chain['fees_3q_cmgr'] = (
                                data_fdf_chain['fees_30_day_avg']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg'].shift(30*3*3)
                                ) ** (1/(3*3)) - 1
data_fdf_chain['fees_1q_cmgr_eth'] = (
                                data_fdf_chain['fees_30_day_avg_eth']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg_eth'].shift(30*3*1)
                                ) ** (1/(3*1)) - 1

data_fdf_chain['fees_3q_cmgr_eth'] = (
                                data_fdf_chain['fees_30_day_avg_eth']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg_eth'].shift(30*3*3)
                                ) ** (1/(3*3)) - 1
# #if focus on op

# display(data_fdf_chain)

In [ ]:
fig = px.line(data_fdf_op, x="date", y="results.oneDayTotalFees", color='metadata.name', title = 'Fees Earned on Optimism (USD)')
fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

fig.write_image(prepend + "img_outputs/svg/app_fees_on_op.svg") #prepend + 
fig.write_image(prepend + "img_outputs/png/app_fees_on_op.png") #prepend + 
fig.write_html(prepend + "img_outputs/app_fees_on_op.html", include_plotlyjs='cdn')

In [ ]:
fig_7d = px.line(data_fdf_op, x="date", y="fees_7_day_avg", color='metadata.name', title = 'Fees Earned on Optimism (USD) - 7 Day Moving Average')
fig_7d.update_layout(yaxis_tickprefix = '$')
# fig_7d.show()

fig_7d.write_image(prepend + "img_outputs/svg/app_fees_on_op_7dma.svg") #prepend + 
fig_7d.write_image(prepend + "img_outputs/png/app_fees_on_op_7dma.png") #prepend + 
fig_7d.write_html(prepend + "img_outputs/app_fees_on_op_7dma.html", include_plotlyjs='cdn')

fig_30d = px.line(data_fdf_op, x="date", y="fees_30_day_avg", color='metadata.name', title = 'Fees Earned on Optimism (USD) - 30 Day Moving Average')
fig_30d.update_layout(yaxis_tickprefix = '$')
# fig_30d.show()

fig_30d.write_image(prepend + "img_outputs/svg/app_fees_on_op_30dma.svg") #prepend + 
fig_30d.write_image(prepend + "img_outputs/png/app_fees_on_op_30dma.png") #prepend + 
fig_30d.write_html(prepend + "img_outputs/app_fees_on_op_30dma.html", include_plotlyjs='cdn')

In [ ]:
fig_chain = px.line(data_fdf_chain, x="date", y="results.oneDayTotalFees", color = "metadata.blockchain", title = 'Sum Fees Earned on apps by Chain (USD)')
fig_chain.update_layout(yaxis_tickprefix = '$')
# fig_chain.show()

fig_chain.write_image(prepend + "img_outputs/svg/app_fees_by_chain.svg") #prepend + 
fig_chain.write_image(prepend + "img_outputs/png/app_fees_by_chain.png") #prepend + 
fig_chain.write_html(prepend + "img_outputs/app_fees_by_chain.html", include_plotlyjs='cdn')

In [ ]:
fig_chain_30d = px.line(data_fdf_chain, x="date", y="results.oneDayTotalFees", color = "metadata.blockchain", title = 'Sum Fees Earned on apps by Chain (USD)')
fig_chain_30d.update_layout(yaxis_tickprefix = '$')
fig_chain_30d.show()

In [ ]:
# ! jupyter nbconvert --to python optimism_app_fees.ipynb